#**DATA CODE**
This code utilizes Spotify APIs to fetch data for individual songs from the "200 Best Christmas Songs" playlist.\
It extracts various musical features, such as:

<div align="center">

| Feature           | Description                  |
|-------------------|------------------------------|
| danceability      | Measure of danceability      |
| energy            | Energy level of the song     |
| key               | Key the track is in          |
| loudness          | Overall loudness of the track|
| mode              | Modality of the track        |
| speechiness       | Speechiness of the track     |
| acousticness      | Acousticness of the track    |
| instrumentalness  | Instrumentalness of the track|
| liveness          | Liveness of the track        |
| valence           | Musical positiveness         |
| tempo             | Tempo of the track           |
| type              | Type of the track            |
| id                | Unique identifier for the track|
| uri               | URI of the track             |
| track_href        | HREF of the track            |
| analysis_url      | URL for audio analysis       |
| duration_ms       | Duration of the track in milliseconds|
| time_signature    | Time signature of the track  |

</div>

In [ ]:
!pip install spotipy --upgrade

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
cid = #codiceid
secret = #psw
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
playlist_link = "https://open.spotify.com/playlist/1vR8ygmTneF5SzdTV7nFUd?si=pL3Bo6fTR0uzOXqcn7LKJQ"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

offset = 0
limit = 100
playlist_tracks = []

while True:
    tracks_batch = sp.playlist_tracks(playlist_URI, limit=limit, offset=offset)["items"]
    if not tracks_batch:
        break
    playlist_tracks.extend(tracks_batch)
    offset += limit

track_uris = [x["track"]["uri"] for x in playlist_tracks]

data = []

for track in playlist_tracks:

    track_uri = track["track"]["uri"]
    track_name = track["track"]["name"]

    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]

    album = track["track"]["album"]["name"]
    track_pop = track["track"]["popularity"]

    audio_features = sp.audio_features(track_uri)[0]

    data.append({
        'Track Name': track_name,
        'Artist Name': artist_name,
        'Artist Popularity': artist_pop,
        'Artist Genres': artist_genres,
        'Album': album,
        'Track Popularity': track_pop,
        'Audio Features': audio_features
    })

df = pd.DataFrame(data)

In [ ]:
audio_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']

for col in audio_features:
    if col in df['Audio Features'][0]:
        df[col] = df['Audio Features'].apply(lambda x: x[col])
    else:
        df[col] = None

df.drop(columns=['Audio Features'], inplace=True)

In [ ]:
df.to_csv('spotify.csv', index = False)